In [16]:
import pandas as pd, os, textwrap, numpy as np

base = "C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data"

print(os.listdir(base))

comorb_raw = pd.read_excel(f"{base}/codige_comorbilità.xlsx")
comorb_tmp = pd.read_excel(f"{base}/codige_tmp_tabella_comorbilità_english.xlsx")
comorb_cat = pd.read_excel(f"{base}/codige_tabella_comorbidità_categoria_english.xlsx")
master = pd.read_excel(f"{base}/codige_english.xlsx")

print("comorb_raw cols:", comorb_raw.columns.tolist())
print("comorb_tmp cols:", comorb_tmp.columns.tolist())
print("comorb_cat cols:", comorb_cat.columns.tolist())
print("master cols (head):", master.columns[:20].tolist())

comorb_cat.head()


['.ipynb_checkpoints', 'codige.xlsx', 'codige_adr_clean.xlsx', 'codige_adr_clean_english.xlsx', 'codige_comorbilità.xlsx', 'codige_english.xlsx', 'codige_ricoveri.xlsx', 'codige_ricoveri_english.xlsx', 'codige_tabella_alterazioni_mutazioni.xlsx', 'codige_tabella_alterazioni_mutazioni_english.xlsx', 'codige_tabella_comorbidità_categoria.xlsx', 'codige_tabella_comorbidità_categoria_english.xlsx', 'codige_tabella_farmaco.xlsx', 'codige_tabella_farmaco_english.xlsx', 'codige_tmp_tabella_comorbilità.xlsx', 'codige_tmp_tabella_comorbilità_english.xlsx', 'codige_trattamento_oncologico.xlsx', 'codige_trattamento_oncologico_english.xlsx', 'Descrizione tabelle ad interim.xlsx', 'FI_lab_score.xlsx', 'master_with_comorbidity.xlsx', 'Untitled.ipynb', 'Untitled1.ipynb', 'verify']
comorb_raw cols: ['id_paziente', 'comorbidita', 'comorbidita_cat', 'altre_pat_n', 'data']
comorb_tmp cols: ['patient_id', 'comorbidita', 'comorbidity_category_list', 'altro', 'data']
comorb_cat cols: ['patient_id', 'co

,patient_id,comorbidita,comorbidity_category_list,altre_pat_n,data
0,1_AO San Pio,diabete tipo II,7,0,2022-05-15
1,1_AO San Pio,ipertensione arteriosa,1,0,2022-03-15
2,1_AO San Pio,patologie cardiovascolari,13,0,2010-10-04
3,1_AORN San Giuseppe Moscati,ipertensione arteriosa,1,0,2022-12-26
4,1_AORN San Giuseppe Moscati,patologie cardiovascolari,13,0,2022-12-26


In [17]:
uniq = comorb_cat["comorbidita"].astype(str).str.strip().str.lower().unique()
len(uniq), uniq[:60]

(16,
 array(['diabete tipo ii', 'ipertensione arteriosa',
        'patologie cardiovascolari', 'dislipidemie', 'obesita', 'bpco',
        'altre patologie', 'insufficienza renale', 'ipb',
        'patologie psichiatriche', 'patologie cerebrovascolari',
        'sindrome depressiva', 'patologie gastrointestinali', 'autoimmune',
        'asma', 'anemia'], dtype=object))

In [19]:
import pandas as pd
from pathlib import Path

base = Path("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data")

# Load comorbidity category table and master dataset
comorb_cat = pd.read_excel(base / "codige_tabella_comorbidità_categoria_english.xlsx")
master = pd.read_excel(base / "codige_english.xlsx")

# Standardize IDs
comorb_cat["patient_id"] = comorb_cat["patient_id"].astype(str).str.strip()
master["patient_id"] = master["patient_id"].astype(str).str.strip()

# Clean comorbidity label
comorb_cat["comorbidita_clean"] = (
    comorb_cat["comorbidita"]
    .astype(str)
    .str.strip()
    .str.lower()
)

unique_terms = sorted(comorb_cat["comorbidita_clean"].unique())
unique_terms

['altre patologie',
 'anemia',
 'asma',
 'autoimmune',
 'bpco',
 'diabete tipo ii',
 'dislipidemie',
 'insufficienza renale',
 'ipb',
 'ipertensione arteriosa',
 'obesita',
 'patologie cardiovascolari',
 'patologie cerebrovascolari',
 'patologie gastrointestinali',
 'patologie psichiatriche',
 'sindrome depressiva']

In [20]:
# Map Italian comorbidities to Charlson condition group (or None)
COMORB_TO_CCI_GROUP = {
    # Weight 1 conditions
    "cardiopatia ischemica": "mi_ihd",              # not in list but kept for safety
    "scompenso cardiaco": "congestive_heart_failure",
    "patologie cardiovascolari": "peripheral_vascular_disease",  # approximate CV disease
    "patologie cerebrovascolari": "cerebrovascular_disease",
    "bpco": "chronic_pulmonary_disease",
    "asma": "chronic_pulmonary_disease",
    "autoimmune": "connective_tissue_disease",
    "ulcera": "ulcer_disease",
    "epatopatia lieve": "mild_liver_disease",
    "diabete tipo ii": "diabetes_without_complications",

    # Weight 2 conditions
    "diabete con complicanze": "diabetes_with_complications",
    "insufficienza renale": "moderate_severe_renal_disease",
    "emiplegia": "hemiplegia",
    "linfoma": "lymphoma",
    "leucemia": "leukemia",
    "tumore maligno pregresso": "any_tumor_non_metastatic",

    # Weight 3 conditions
    "epatopatia moderata severa": "moderate_severe_liver_disease",

    # Weight 6 conditions
    "metastasi": "metastatic_solid_tumor",
    "aids": "aids_hiv",
    "hiv": "aids_hiv",
}

# Charlson weights per group
CCI_WEIGHTS = {
    # Weight 1
    "mi_ihd": 1,
    "congestive_heart_failure": 1,
    "peripheral_vascular_disease": 1,
    "dementia": 1,
    "cerebrovascular_disease": 1,
    "chronic_pulmonary_disease": 1,
    "connective_tissue_disease": 1,
    "ulcer_disease": 1,
    "mild_liver_disease": 1,
    "diabetes_without_complications": 1,

    # Weight 2
    "diabetes_with_complications": 2,
    "hemiplegia": 2,
    "moderate_severe_renal_disease": 2,
    "any_tumor_non_metastatic": 2,
    "leukemia": 2,
    "lymphoma": 2,

    # Weight 3
    "moderate_severe_liver_disease": 3,

    # Weight 6
    "metastatic_solid_tumor": 6,
    "aids_hiv": 6,
}

# Assign CCI group to each comorb entry
def map_to_cci_group(name: str):
    name = name.lower().strip()
    if name in COMORB_TO_CCI_GROUP:
        return COMORB_TO_CCI_GROUP[name]
    # no direct mapping
    return None

comorb_cat["cci_group"] = comorb_cat["comorbidita_clean"].apply(map_to_cci_group)

# Inspect mapping results
mapped_counts = comorb_cat["cci_group"].value_counts(dropna=False)
mapped_counts


cci_group
None                              344
peripheral_vascular_disease        75
diabetes_without_complications     59
chronic_pulmonary_disease          32
moderate_severe_renal_disease       9
connective_tissue_disease           8
cerebrovascular_disease             4
Name: count, dtype: int64

In [21]:
# Compute CCI per patient from comorb_cat with cci_group

# Drop rows without CCI group
cci_rows = comorb_cat.dropna(subset=["cci_group"]).copy()

# Aggregate unique groups per patient
patient_groups = (
    cci_rows
    .groupby("patient_id")["cci_group"]
    .apply(lambda x: sorted(set(x)))
    .rename("cci_groups")
    .to_frame()
)

def compute_cci_score(groups):
    if not isinstance(groups, (list, tuple)):
        return 0
    score = 0
    for g in set(groups):
        score += CCI_WEIGHTS.get(g, 0)
    return score

patient_groups["cci_score"] = patient_groups["cci_groups"].apply(compute_cci_score)

# Merge onto master (codige_english)
cci = patient_groups.reset_index()
master_cci = master.merge(cci, on="patient_id", how="left")

# For patients with no comorbidity data, CCI = 0
master_cci["cci_score"] = master_cci["cci_score"].fillna(0).astype(int)

out_cci_scores = base / "cci_scores.xlsx"
out_master_cci = base / "codige_with_cci.xlsx"

cci.to_excel(out_cci_scores, index=False)
master_cci.to_excel(out_master_cci, index=False)

(out_cci_scores, out_master_cci, master_cci["cci_score"].value_counts().sort_index())

C:\Users\HP\AppData\Local\Temp\ipykernel_26908\506949139.py:35: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  cci.to_excel(out_cci_scores, index=False)
C:\Users\HP\AppData\Local\Temp\ipykernel_26908\506949139.py:36: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  master_cci.to_excel(out_master_cci, index=False)


(WindowsPath('C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/cci_scores.xlsx'),
 WindowsPath('C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/codige_with_cci.xlsx'),
 cci_score
 0    271
 1     89
 2     36
 3      7
 4      3
 Name: count, dtype: int64)

In [22]:
master_cci = master.merge(cci, on="patient_id", how="left")
master_cci["cci_score"] = master_cci["cci_score"].fillna(0).astype(int)